In [ ]:
# os is used for Ilka's linux :)
#import os
#os.environ["QT_QPA_PLATFORM"] = "xcb"

import cv2
from ultralytics import YOLO


In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolo11n.yaml")  # build a new model from YAML
# model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="./dataset.yaml", epochs=50, imgsz=640, batch=4)


and now predict

In [1]:
from ultralytics import YOLO
import cv2
import math



# Carga del modelo
model_objects = YOLO("yolo11n.pt")
model_lipl = YOLO("./datasets/selfmadeYOLO/runs/detect/50-epochs/weights/best.pt")
video_path = 'test-vid.mp4'  
output_path = 'anonymized_output.mp4'


# Etiqueta de las distintas clases
classNames = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
    "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
    "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush", "license plate"  # added license plate here
]

# Captura desde el archivo de video
vid = cv2.VideoCapture(video_path)

# Configuración de salida de video anonimizado
fps = int(vid.get(cv2.CAP_PROP_FPS))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while True:
    ret, img = vid.read()
    
    if not ret:
        break

    # Detecta en la imagen
    res_obj = model_objects.track(img)
    res_lipl = model_lipl.track(img)

    # annotate frame
    img = res_obj[0].plot() 
    img = res_lipl[0].plot(img=img)

    # apply object detection
    for r in res_obj:
        boxes = r.boxes

        for box in boxes:
            # Obtén las coordenadas del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros

            # Obtén la clase detectada
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            # Aplicar desenfoque en áreas específicas
            if detected_class == "person":
                # Aplicar desenfoque en el tercio superior de la persona (cara)
                face_region = img[y1:y1 + (y2 - y1) // 3, x1:x2]
                blurred_face = cv2.GaussianBlur(face_region, (51, 51), 30)
                img[y1:y1 + (y2 - y1) // 3, x1:x2] = blurred_face  # Reemplazar con área desenfocada

    # apply license plate model
    for r in res_lipl: 

        boxes = r.boxes
        for box in boxes:
            # Obtén las coordenadas del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros

            # Obtén la clase detectada
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            #if detected_class == "license plate":
                # Aplicar desenfoque en la región de la matrícula
            plate_region = img[y1:y2, x1:x2]
            blurred_plate = cv2.GaussianBlur(plate_region, (51, 51), 30)
            img[y1:y2, x1:x2] = blurred_plate  # blurr

    out.write(img)

    # show video 
    cv2.imshow('privacy', img)

    # Salir con ESC
    if cv2.waitKey(20) == 27:
        break

# Libera recursos
vid.release()
out.release()
cv2.destroyAllWindows()


OpenCV: FFMPEG: tag 0x3476706d/'mpv4' is not found (format 'mp4 / MP4 (MPEG-4 Part 14)')'



0: 384x640 1 license plate, 198.7ms
Speed: 4.8ms preprocess, 198.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jeski/sem/vc/Visi-n-por-Computador/.venv/lib/python3.12/site-packages/cv2/qt/plugins"


0: 384x640 1 license plate, 216.1ms
Speed: 17.5ms preprocess, 216.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 103.2ms
Speed: 3.1ms preprocess, 103.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 163.6ms
Speed: 2.8ms preprocess, 163.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 86.7ms
Speed: 3.9ms preprocess, 86.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 83.9ms
Speed: 3.2ms preprocess, 83.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 88.6ms
Speed: 4.4ms preprocess, 88.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 88.7ms
Speed: 2.5ms preprocess, 88.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 91.8ms
Speed: 10.5ms preprocess, 9

: 

test with tracking


In [3]:
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("./runs/detect/50-epochs/weights/best.pt")  # Replace with your model path

# Specify the input and output video paths
video_path = "/home/jeski/sem/vc/test-vid-prof.mp4"  # Replace with your input video path
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4"  # Path to save the annotated video

# Open the input video file
cap = cv2.VideoCapture(video_path)

# Get video properties for output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

# Iterate through each frame in the video for tracking
for results in model.track(source=video_path, stream=True):
    # Get the annotated frame
    annotated_frame = results.plot()  # Adds bounding boxes and labels to the frame

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Release the resources
cap.release()
out.release()

print(f"Annotated video saved to {output_path}")



video 1/1 (frame 1/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 (no detections), 140.5ms
video 1/1 (frame 2/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 1 license plate, 83.1ms
video 1/1 (frame 3/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 1 license plate, 98.3ms
video 1/1 (frame 4/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 1 license plate, 97.8ms
video 1/1 (frame 5/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 3 license plates, 78.8ms
video 1/1 (frame 6/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 2 license plates, 81.2ms
video 1/1 (frame 7/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 1 license plate, 72.2ms
video 1/1 (frame 8/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 1 license plate, 81.9ms
video 1/1 (frame 9/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 1 license plate, 72.9ms
video 1/1 (frame 10/2832) /home/jeski/sem/vc/test-vid-prof.mp4: 384x640 1 license plate, 68.9ms
video 1/1 (frame 11/2832) /home/jeski/sem/vc/

test with image


In [3]:
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/train/weights/best.pt")  # Replace with the path to your trained model

# Load the image
image_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/Our-Images/train/images/1729779190271.jpg"  # Replace with the path to your image
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print("Error: Could not load image.")
    exit()

# Run YOLO model on the image
results = model(image)

# Visualize the results on the image
annotated_image = results[0].plot()  # plot() adds bounding boxes and labels to the image

# Display the result
#cv2.imshow("YOLO Detection", annotated_image)

# Wait for a key press to close the window
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Save the result (optional)
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-1.jpg"  # Specify where to save the result
cv2.imwrite(output_path, annotated_image)
print(f"Annotated image saved to {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/train/weights/best.pt'

test with blurring people and license plates

In [7]:
from ultralytics import YOLO
import cv2
import math



# Carga del modelo
model_objects = YOLO("yolo11n.pt")
model_lipl = YOLO("./runs/detect/50-epochs/weights/best.pt")
video_path = "/home/jeski/sem/vc/test-vid-prof.mp4"  
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof-2.mp4"


# Etiqueta de las distintas clases
classNames = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
    "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
    "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush", "license plate"  # Añadir 'license plate' aquí
]

# Captura desde el archivo de video
vid = cv2.VideoCapture(video_path)

# Configuración de salida de video anonimizado
fps = int(vid.get(cv2.CAP_PROP_FPS))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while True:
    ret, img = vid.read()
    
    if not ret:
        break

    # Detecta en la imagen
    res_obj = model_objects(img, stream=True)
    res_lipl = model_lipl(img, stream=True)

    print("res type: ", type(res_obj))

    # apply object detection
    for r in res_obj:
        boxes = r.boxes

        for box in boxes:
            # Obtén las coordenadas del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros

            # Obtén la clase detectada
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            # Aplicar desenfoque en áreas específicas
            if detected_class == "person":
                # Aplicar desenfoque en el tercio superior de la persona (cara)
                face_region = img[y1:y1 + (y2 - y1) // 3, x1:x2]
                blurred_face = cv2.GaussianBlur(face_region, (51, 51), 30)
                img[y1:y1 + (y2 - y1) // 3, x1:x2] = blurred_face  # Reemplazar con área desenfocada

    # apply license plate model
    for r in res_lipl:
        boxes = r.boxes

        for box in boxes:
            # Obtén las coordenadas del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros

            # Obtén la clase detectada
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            #if detected_class == "license plate":
                # Aplicar desenfoque en la región de la matrícula
            plate_region = img[y1:y2, x1:x2]
            blurred_plate = cv2.GaussianBlur(plate_region, (51, 51), 30)
            img[y1:y2, x1:x2] = blurred_plate  # Reemplazar con área desenfocada

    # Escribe el fotograma anonimizado en el archivo de salida
    out.write(img)

    # Opcional: muestra el video en tiempo real
    #cv2.imshow('Anonimizado', img)

    # Salir con ESC
    if cv2.waitKey(20) == 27:
        break

# Libera recursos
vid.release()
out.release()
cv2.destroyAllWindows()


res type:  <class 'generator'>

0: 384x640 4 cars, 1 bus, 77.9ms
Speed: 3.2ms preprocess, 77.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 90.8ms
Speed: 7.0ms preprocess, 90.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
res type:  <class 'generator'>

0: 384x640 4 cars, 1 bus, 120.3ms
Speed: 3.2ms preprocess, 120.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 96.7ms
Speed: 4.0ms preprocess, 96.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
res type:  <class 'generator'>

0: 384x640 1 person, 5 cars, 1 bus, 118.5ms
Speed: 2.9ms preprocess, 118.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 124.5ms
Speed: 3.3ms preprocess, 124.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
res type:  <class 'generator'>

0: 384x640 5 cars, 1 bus, 87.9ms
Speed: 4.0ms preprocess, 87.9ms in